In [1]:
!pip install tensorflow opencv-python


In [9]:
import tensorflow as tf
import os

model_dir = "ssd_mobilenet_v2_fpnlite_320x320"
os.makedirs(model_dir, exist_ok=True)

# Download the model
model_url = "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz"
tf.keras.utils.get_file(
    fname=f"{model_dir}.tar.gz",
    origin=model_url,
    extract=True,
    cache_dir=".",
    cache_subdir=model_dir,
)


'./ssd_mobilenet_v2_fpnlite_320x320/ssd_mobilenet_v2_fpnlite_320x320.tar.gz'

In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import os

# Load the model
model_dir = "ssd_mobilenet_v2_fpnlite_320x320/saved_model"
detect_fn = tf.saved_model.load(model_dir)

# Load labels
labels_path = "ssd_mobilenet_v2_fpnlite_320x320/mscoco_label_map.pbtxt"
category_index = {}
with open(labels_path, 'r') as file:
    for line in file:
        if "id:" in line:
            class_id = int(line.split(":")[1].strip())
        if "display_name:" in line:
            class_name = line.split(":")[1].strip().replace("'", "")
            category_index[class_id] = class_name

def load_image_into_numpy_array(path):
    return np.array(cv2.imread(path))

def run_inference(image):
    input_tensor = tf.convert_to_tensor(image)
    input_tensor = input_tensor[tf.newaxis, ...]
    detections = detect_fn(input_tensor)

    return detections

def draw_boxes(image, detections, threshold=0.5):
    count = 0
    height, width, _ = image.shape
    for i in range(detections['detection_scores'].shape[1]):
        score = detections['detection_scores'][0][i].numpy()
        if score >= threshold:
            count += 1
            bbox = detections['detection_boxes'][0][i].numpy()
            ymin, xmin, ymax, xmax = bbox
            left, right, top, bottom = (xmin * width, xmax * width,
                                        ymin * height, ymax * height)
            class_id = int(detections['detection_classes'][0][i].numpy())
            label = category_index[class_id]
            color = (0, 255, 0)
            cv2.rectangle(image, (int(left), int(top)), (int(right), int(bottom)), color, 2)
            cv2.putText(image, label, (int(left), int(top) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    return count, image

if __name__ == "__main__":
    image_path = 'path/to/your/image.jpg'
    image_np = load_image_into_numpy_array(image_path)

    detections = run_inference(image_np)
    count, result_image = draw_boxes(image_np, detections)

    print(f"Number of items: {count}")
    cv2.imshow('Result', result_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
